# Notebook for Working with Solaris Zones Through its REST API — Basic Actions

This Jupyter Notebook is aimed at showing how you can use the REST interface in Oracle Solaris to work with [Solaris Zones](https://docs.oracle.com/cd/E37838_01/html/E61037/zonesoverview.html) (a.k.a. Solaris Containers). The REST API is layered on top of the [Oracle Solaris Remote Administration Deamon (RAD)](https://docs.oracle.com/cd/E37838_01/html/E68270/gpzxz.html#scrolltoc), and gives access to all the RAD modules through REST. This notebook is using Python to run all the tasks. </br></br>
In this notebook we'll show how to connect to the REST interface and how to perform various tasks you can do through the REST API.

> *Note 1* — You'll need to enable the `svc:/system/rad:remote` [service](https://docs.oracle.com/cd/E37838_01/html/E68270/gpzpd.html#scrolltoc) to be able to remotely connect. </br> 
> *Note 2* — If the server doesn't have a certification signed by a public CA then pull in `/etc/certs/localhost/host-ca/hostca.crt` from your server and point to its location in the JSON file below. </br> 
> *Note 3* — To do actions beyond looking at zones and their configuration, like creating, installing, booting, and deleting a zone, the user you use to connect to the system will need to have either the `Zone Management` and/or `Zone Configuration` RBAC profiles set to be able perform certain tasks. For more info [see *Using Rights Profiles to Install and Manage Zones*](https://docs.oracle.com/cd/E37838_01/html/E61039/zadmt-rightsprofiles.html#scrolltoc).</br>
> *Note 4* — This notebook was written with Python 3.7

Once these things are set you're ready to start.

## Introduction to the Solars Zones API

The base URI of the Solaris Zones API is: `api/com.oracle.solaris.rad.zonemgr/1.7/`

The API has five main interfaces:

- **Zone** — URI: `Zone/` — For operations that affect a single zone, it represents an individual zone. All zone configuration and administrative actions are represented in this interface. 
- **ZoneManager** — URI: `ZoneManager/` — Manage the zones on this system as whole, for example creating and deleting zones.
- **ZoneInfo** — URI: `ZoneInfo/` — Report on the zone from which the RAD/REST service is running. It will give basic information about this zone. This interface is read only.
- **MigrationConnection** — URI: `MigrationConnection/` — Information about the migration connection to a particular host. This interface is mostly read only.
- **ZoneMigration** — URI: `ZoneMigration/` — This interface represents an ongoing zone migration. A migration may be cancelled at an arbitrary point, for example if the zone is halted and deleted.

In general the base interactions with the system when a zone is not specified are done through the `ZoneManager/` interface (for example to get the list of zone or to create a zone), and all the interactions with a specific zone are done through the `Zone/` interface.

For the full list of methods for each interface see the [Appendix](#Appendix) at the bottom of the notebook. For documentation on the complete API please install the `system/management/webui/webui-docs` package and access it through the WebUI.

Because there are many different things you can do with the Solaris Zones module, this notebook will focus on some basic steps around Solaris Zones management. This includes getting the list of the zones on the system and getting their configuration settings, as well as creating, installing, booting, halting/shutting down, uninstalling, and deleting zones.

Other notebooks will cover more advanced topics.

---

## Imports and Setting Variables

First to import all the Python libraries:

In [1]:
import requests
import json
import base64
import os
import time, datetime
import pandas as pd

Turning off warnings thrown by Python:

In [2]:
import warnings
warnings.filterwarnings('ignore')

---
## Base Functions

Next we define the base functions used to make the connection with the REST interface. The first is to establish a session, the second is for the regular GET, PUT, POST, and DELETE methods.

The function to establish a session:

In [3]:
def rad_rest_login(session_name, server_connection_info, base_authentication_uri):
    login_url = 'https://{0}:{1}/{2}'.format(*list(map(server_connection_info.get, ['server_name', 'server_port'])), base_authentication_uri)
    print("Logging in with this URL: " + login_url)

    try:
        response = session_name.post(login_url, json = server_connection_info['config_json'], verify = server_connection_info['cert_location'])
    except:
        print('no connection')
        response = 'empty'
    
    return response

The function to run regular requests:

In [4]:
def rad_rest_request(request_type, session_name, server_connection_info, rad_rest_uri, payload = None):
    
    # Create a list of variables
    query_vars = list(map(server_connection_info.get, ['server_name', 'server_port']))
    query_vars.append(rad_rest_uri)

    # Create query url to be used
    query_url = 'https://{0}:{1}/{2}'.format(*query_vars)

    response = session_name.request(request_type, query_url, json = payload)
    
    return response

Two extra functions for convenience to convert Solaris timestamps to something human-readable, which might be useful later on:

In [5]:
def timestamp_from_solaris_time(value):
    return datetime.datetime.fromtimestamp(value/1000)

In [6]:
def str_from_solaris_time(value):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value/1000))

---

## Load Remote System Connection Information

Now define the location of the remote system connection information:

In [7]:
config_filename = '../base_login_zone.json'

The system specific information is located in a JSON file located in the same directory as the notebook, and is structured like this: </br>

```json
{
    "server_name": "<ip address>",
    "server_port": "<RAD Remote port>",
    "cert_location": "<path_to_cert>",
    "config_json": {
        "username": "<username>",
        "password": "<password>",
        "scheme": "pam",
        "preserve": true,
        "timeout": -1
    }
}
``` 

We advise to securely connect to the remote server over HTTPS however this will require the use of a certificate, in many cases the remote server will have a certificate signed by a public CA, if so please set `"cert_location": true`. If, for example in the case of a demo system, the certificate isn't signed, download the `/etc/certs/localhost/host-ca/hostca.crt` file from the system you're connecting to and refer to its local location with `"cert_location": "<path_to_cert>"`. If you don't want to use a certificate at all set `"cert_location": false` instead of the location of the cert file. Note this last option doesn't validate that you're connecting to correct server.

Loading this system specific information:

In [8]:
if config_filename:
    with open(config_filename, 'r') as f:
        server_connection_info = json.load(f)
        
server_connection_info['server_name']

't8ldom5.us.oracle.com'

---
---
## Connecting to the Solaris Zones Interface

In this notebook we'll run through the following basic example steps:

* [**Getting the information about the zone you're connecting to**](#Getting-the-Information-about-the-Zone) — You could be connecting to a Global Zone (including an LDom or a Kernel Zone) or a Native Zone, this shows what type of information you'll get.
* [**Getting a list of the zones**](#Getting-a-List-of-the-Zones)  — This will show the zones nested in the instance you're connecting to.
* [**Getting the zone configuration**](#Getting-a-Zone-Configuration) — This shows how to get the configuration data for a specific zone in the list.
* [**Managing the zone state and lifecycle**](#Managing-the-Zone-State-and-Lifecycle)
    * [**Booting and halting zones**](#Booting-and-Halting-Zones) — This shows how to check what the state is of the zones, and how to boot and halt a zone.
    * [**Creating and deleting a zone**](#Creating-and-Deleting-Zones) — This shows how to create a new zone, how to define it, how to install it and how to uninstall and delete it again.

We start with the example on how to connect to the server to get the list of the zones on the system and work from there. 

### Defining Interface URIs

Before we start we define the various URI variables:

In [9]:
base_authentication_uri = 'api/authentication/1.0/Session/'

base_zone_uri = 'api/com.oracle.solaris.rad.zonemgr/1.7/'
zone_manager_uri = '{}ZoneManager/'.format(base_zone_uri)
zone_uri = '{}Zone/'.format(base_zone_uri)
zone_info_uri = '{}ZoneInfo/'.format(base_zone_uri)
migration_connection_uri = '{}MigrationConnection/'.format(base_zone_uri)
zone_migration_uri = '{}ZoneMigration/'.format(base_zone_uri)

**Note:** In this notebook we won't be using the last two URIs, something for a later notebook, but we've included them for completeness. 

### Logging in and setting up a session

First establish the session and bring in the necessary credentials. Note, we're printing the results to give a better insight on what's coming back, we won't printing all the information comming back from the request just certain parts to give you a feel for it:

In [10]:
s = requests.Session()

login_answer = rad_rest_login(s, server_connection_info, base_authentication_uri)

print(login_answer.status_code, '\n')
print(login_answer.text)

Logging in with this URL: https://t8ldom5.us.oracle.com:6788/api/authentication/1.0/Session/
201 

{
        "status": "success",
        "payload": {
                "href": "/api/com.oracle.solaris.rad.authentication/1.0/Session/_rad_reference/5632"
        }
}


---
### Getting the Information about the Zone

Before looking at the list of zones, we want to quickly look at the info you can get about the current zone you're connecting to. 

To do this we'll use the `GET` method with the `ZoneInfo` interface and use `?_rad_detail` to query it:<br>
`HTTPS://{{server}}:6788/api/com.oracle.solaris.rad.zonemgr/1.7/ZoneInfo/?_rad_detail`

This will return the status of the request, and if it succeded the payload will return the `href` and the `ZoneInfo`:

In [11]:
zone_info = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(zone_info_uri))
zone_info_text = json.loads(zone_info.text)

print(zone_info.status_code, '\n')
print(zone_info_text)

200 

{'status': 'success', 'payload': {'href': 'api/com.oracle.solaris.rad.zonemgr/1.7/ZoneInfo', 'ZoneInfo': {'brand': 'solaris', 'id': 0, 'uuid': None, 'name': 'global', 'isGlobal': True}}}


As you can see this will return the `brand`, the `id`, the `uuid`, the `name`, and `isGlobal` values for the zone we're connected to. If in this case this the Global Zone, we're getting all the values you'd expect in that case, like the `id` is `0`, the `name` is `global` and the `isGlobal` is `True`.

Also note that this information isn't in the list when using the `Zone/` interface, that interface only shows its "guest" zones. This is different than the CLI `zoneadm list -cv` command which will also list the Global Zone.

---
### Getting a List of the Zones

Now we'll get the list of Solaris Zones use `GET` mothod on the `Zone` interface again using `?_rad_detail` to query it:<br>
`HTTPS://{{server}}:6788/api/com.oracle.solaris.rad.zonemgr/1.7/Zone/?_rad_detail`

This will return a list of dicts, one for each zone containing basic information about the given zone:

In [12]:
zones_list = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(zone_uri))
zones_list_text = json.loads(zones_list.text)

print(zones_list.status_code, '\n')
print(zones_list_text)

200 

{'status': 'success', 'payload': [{'href': 'api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test1', 'Zone': {'auxstate': [], 'brand': 'solaris', 'id': -1, 'uuid': 'c5f26d28-0a9d-4c20-a2f3-eb2225de7f7e', 'name': 'test1', 'state': 'installed'}}]}


It's not very easy to read and work with this response so we load it into a Pandas DataFrame which is much easier to read:

In [13]:
zones_list_df = pd.io.json.json_normalize(zones_list_text['payload'])
zones_list_df

,href,Zone.auxstate,Zone.brand,Zone.id,Zone.uuid,Zone.name,Zone.state
0,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test1,[],solaris,-1,c5f26d28-0a9d-4c20-a2f3-eb2225de7f7e,test1,installed


Note again that this doesn't contain an entry for the zone you're connected to.

As you can see I'm not using using the usual `DataFrame` function to load the data but instead `io.json.json_normalize` to flatten the nested dictionaires into a single dataframe.

Now it's in a Pandas DataFrame you can quickly search for certain things, like for example which zones are currently running (by using `.str.contains('running')` for the values in `Zone.state`):

In [14]:
zones_list_df[zones_list_df['Zone.state'].str.contains('running')]

,href,Zone.auxstate,Zone.brand,Zone.id,Zone.uuid,Zone.name,Zone.state


Important, if you want to do something with a specific zone you can use the value in the `href` field. The structure of the resulting URI when using this with a RAD method is like this:<br>
`api/com.oracle.solaris.rad.zonemgr/1.7/Zone/{{zonename}}/_rad_method/{{RAD_method}}`

How to use it as we'll show now.

---
### Getting a Zone Configuration

We'll now look at getting the configuration of the first zone in the list. To do this we pull the URI from the `href` field in the first row of the dataframe and combine this with the `exportConfig` RAD method. This is done with a `PUT` method which requires we include JSON text, and in this case we want to make sure we could use this result later to create a new zone so we use the JSON text `{'reimport': True}`:

In [15]:
single_zone_uri = zones_list_df['href'][0]

json_body = {'reimport': True}

single_zone_config = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/exportConfig'.format(single_zone_uri), json_body)
single_zone_config_text = json.loads(single_zone_config.text)

print(single_zone_config.status_code, '\n')
print(single_zone_config_text)

200 

{'status': 'success', 'payload': 'create -Fb\nset brand=solaris\nadd anet\nset linkname=net0\nset configure-allowed-address=true\nend\n'}


The response contains the `status` and the `payload`. To better read the response print out the contents of `payload`:

In [16]:
print(single_zone_config_text['payload'])

create -Fb
set brand=solaris
add anet
set linkname=net0
set configure-allowed-address=true
end



This is essentially is the same content as the `zonecfg -z <zonename> export` CLI command. Note that the export produces output in a form suitable for use in a CLI command file and includes only non-default values explicitly set by the user.

If for example we'd want to use this to create another on the basis of this zone, we can save this payload to use it at a later point in time.

Of course the actual zone configuration is much longer, probably with a bunch of values not being set. To get the full list of the actual zone use the `PUT` method with the same URI for the zone and the use the `getResources` RAD method. 

The result looks like this:

In [17]:
json_body = {}

single_zone_res = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/getResources'.format(single_zone_uri), json_body)
single_zone_res_text = json.loads(single_zone_res.text)

print(single_zone_res.status_code, '\n')
print(single_zone_res_text['payload'])

200 

[{'type': 'global', 'properties': [{'name': 'zonename', 'value': 'test1', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'zonepath', 'value': '/system/zones/%{zonename}', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'brand', 'value': 'solaris', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'autoboot', 'value': 'false', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'autoshutdown', 'value': 'shutdown', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'bootargs', 'value': '', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'file-mac-profile', 'value': '', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'pool', 'value': '', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'limitpriv', 'value': '', 'type': 'PROP_SIMPLE', 'listvalue': None, 'complexvalue': None}, {'name': 'sc

Again this is hard to read and work with so here too we'll pull it into a Pandas DataFrame.

Note that the `payload` is a list of nested dicts, one for each type of `Resource`, so we need to do a little bit more work to nicely pull it all into the single dataframe. Plus I add a column that holds the name of the `resoucetype` and the `parent` of the `Resource`:

In [18]:
full_single_zone_res_df = pd.DataFrame()

for index, entry in enumerate(single_zone_res_text['payload']):
    single_zone_res_df = pd.io.json.json_normalize(single_zone_res_text['payload'][index]['properties'])
    single_zone_res_df['resourcetype'] = entry['type']
    single_zone_res_df['parent'] = entry['parent']
    full_single_zone_res_df = pd.concat([full_single_zone_res_df, single_zone_res_df])

full_single_zone_res_df

,name,value,type,listvalue,complexvalue,resourcetype,parent
0,zonename,test1,PROP_SIMPLE,None,None,global,None
1,zonepath,/system/zones/%{zonename},PROP_SIMPLE,None,None,global,None
2,brand,solaris,PROP_SIMPLE,None,None,global,None
3,autoboot,false,PROP_SIMPLE,None,None,global,None
4,autoshutdown,shutdown,PROP_SIMPLE,None,None,global,None
5,bootargs,,PROP_SIMPLE,None,None,global,None
6,file-mac-profile,,PROP_SIMPLE,None,None,global,None
7,pool,,PROP_SIMPLE,None,None,global,None
8,limitpriv,,PROP_SIMPLE,None,None,global,None
9,scheduling-class,,PROP_SIMPLE,None,None,global,None


**Note:** Some of these properties actually are lists of properties. And also most of these properties are not set to anything, that is to say they are either empty values or an empty list `[]`. 

To filter those out we can remove the empty lists (by looking for `.str.len()` not being zero for each list) and then the empty values:

In [19]:
full_single_zone_res_df = full_single_zone_res_df[full_single_zone_res_df['listvalue'].str.len() != 0]
full_single_zone_res_df[full_single_zone_res_df['value'] != '']

,name,value,type,listvalue,complexvalue,resourcetype,parent
0,zonename,test1,PROP_SIMPLE,None,None,global,None
1,zonepath,/system/zones/%{zonename},PROP_SIMPLE,None,None,global,None
2,brand,solaris,PROP_SIMPLE,None,None,global,None
3,autoboot,false,PROP_SIMPLE,None,None,global,None
4,autoshutdown,shutdown,PROP_SIMPLE,None,None,global,None
10,ip-type,exclusive,PROP_SIMPLE,None,None,global,None
14,global-time,false,PROP_SIMPLE,None,None,global,None
15,boot-priority,normal,PROP_SIMPLE,None,None,global,None
0,linkname,net0,PROP_SIMPLE,None,None,anet,None
1,lower-link,auto,PROP_SIMPLE,None,None,anet,None


**Note:** The way we built the dataframe resulted in each property having its own index. This means you can either grab a specific row using `.iloc[]`, in this case the first row:

In [20]:
full_single_zone_res_df.iloc[0]

name               zonename
value                 test1
type            PROP_SIMPLE
listvalue              None
complexvalue           None
resourcetype         global
parent                 None
Name: 0, dtype: object

Or you can the a certain row from each property using `.loc[]`, for example the first row of each:

In [21]:
full_single_zone_res_df.loc[0]

,name,value,type,listvalue,complexvalue,resourcetype,parent
0,zonename,test1,PROP_SIMPLE,None,None,global,None
0,linkname,net0,PROP_SIMPLE,None,None,anet,None


For this notebook we wanted to only show how to get configuration values of a specific zone, changing the values and making these changes active is for another time. 

---
### Managing the Zone State and Lifecycle

Now we're going to look at how to manage an individual zone, how to get its current state and how to manage its lifecycle. I.e. how to create/boot/halt/delete a zone.

#### Booting and Halting Zones

First we'll use the zone we've been working with up to now and get its state. Again using the same URI we pulled from the `href` column, adding `?_rad_detail`, and pulling the `state` from the `payload` and `Zone` dicts:

In [22]:
zones_status = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(single_zone_uri))
print(json.loads(zones_status.text)['payload']['Zone']['state'])

installed


For this example we're expecting the result to be `installed` as this will allow us to then boot the zone. 

If the `state` is indeed `installed` then we can use the `PUT` method in combination with the `boot` RAD method (equivalent to `zoneadm -z <zonename> boot`), again sending an empty JSON text with it:

In [23]:
json_body = {}

single_zone_boot = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/boot'.format(single_zone_uri), json_body)

print(single_zone_boot.status_code, '\n')
print(json.loads(single_zone_boot.text))

200 

{'status': 'success', 'payload': {'code': 'NONE', 'str': '', 'stdout': None, 'stderr': None}}


If all is well this will take a few moments and should return a `200` and `'status': 'success'`. 

If not, for example if the zone is already running it will return a `503`, and the `stterr` in the `payload` will say something like `error: zone is already booted\nzoneadm: zone test1: call to zoneadmd(8) failed: zoneadmd(8) returned an error 5 (operation not allowed in the current zone state)\n`. Again identical to the response you'd get on the CLI `zoneadm -z <zonename> boot` command.

The JSON text will allow you to send boot options if you'd like. These options would be expressed in a list of strings, for example if you'd like to force a suspended Kernel Zone to do a new boot and come up to single user mode you'd send this:
```json
{
    'options': [
        '-R', '-s' 
    ]
}
```

But for this case we're just booting an installed zone with no options, so we only send:
```json
{}
```

We can check if the zone is now running by again looking at the `state` in the respose of the `?_rad_detail` call:

In [24]:
zones_status = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(single_zone_uri))
print(json.loads(zones_status.text)['payload']['Zone']['state'])

running


This should now return `running`.

For this example we won't do anything with the zone at the moment so we'll halt it using the `halt` RAD method which is equivalent to the `zoneadm -z <zonename> halt` CLI command. 

Normally you'd probably want to use the `shutdown` RAD method for a cleaner shut down of the zone.

This is done with the `PUT` method and only requires the empty JSON text to be sent along as we don't need extra options:

In [25]:
json_body = {}

single_zone_halt = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/halt'.format(single_zone_uri), json_body)

print(single_zone_halt.status_code, '\n')
print(json.loads(single_zone_halt.text))

200 

{'status': 'success', 'payload': {'code': 'NONE', 'str': '', 'stdout': None, 'stderr': None}}


This again should return as `success` indicating the `halt` command completed, bringing the zone state back to `installed`.

Again we check if this is indeed the case:

In [26]:
zones_status = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(single_zone_uri))
print(json.loads(zones_status.text)['payload']['Zone']['state'])

installed


And the zone is back in the `installed` state.

---
#### Creating and Deleting Zones

Next we'll show how a zone can be created and installed from scratch. This is a bit more involved as you don't have a zone to start with, so there's nothing to refer to when using the `Zone/` interface. 

In order to do this first step and create a new zone using the `ZoneManager/` interface together with the `create` or `importConfig` RAD methods. The first is equivalent to the plain `zonecfg -z <zonename> create` CLI command, and the second is equivalent to the `zonecfg -z <zonename> -f <command_file>` CLI command. Where the `<command_file>` is the list of `zonecfg` commands.

When using `create` you'll get a "vanilla" zone, with no customizations so the JSON text would be empty, and when using `importConfig` you'll have to supply it with a string containing the `zonecfg` commands in the same format as the output from the `exportConfig` method we used at the top of this notebook.

Either method uses a `PUT` method and will require at least the name of the new zone so we define this first:

In [27]:
new_zone = 'test2'

In this example we'll be using the `importConfig` RAD method. In this case the `autoboot` will also be set to `true` in addition to the base template settings:

In [28]:
zone_config = 'create -b\nset brand=solaris\nset autoboot=true\nadd anet\nset linkname=net0\nset configure-allowed-address=true\nend\n'

When using `importConfig` there is also the `'noexecute'` boolean value that has to be set as part of the JSON text. If `'noexecute'` is set to `True` the `importConfig` will only verify the given command file contents.

After this we can go ahead and import the config and thereby create the new zone:

In [29]:
json_body = {'configuration': [zone_config], 'name': new_zone, 'noexecute': False}

single_zone_create = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/importConfig'.format(zone_manager_uri), json_body)

print(single_zone_create.status_code, '\n')
print(json.loads(single_zone_create.text))

200 

{'status': 'success', 'payload': {'code': 'NONE', 'str': '', 'stdout': None, 'stderr': None}}


This should return the status `success`, and now we can check if it's now in the list of zones:

In [30]:
zones_list = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(zone_uri))
print(zones_list.status_code)

zones_list_df = pd.io.json.json_normalize(json.loads(zones_list.text)['payload'])
zones_list_df

200


,href,Zone.auxstate,Zone.brand,Zone.id,Zone.uuid,Zone.name,Zone.state
0,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test1,[],solaris,-1,c5f26d28-0a9d-4c20-a2f3-eb2225de7f7e,test1,installed
1,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test2,[],solaris,-1,None,test2,configured


This should show the state of the new zone as `configured`.

To further work with this new zone we'll pull its `href` from the dataframe:

In [31]:
single_zone_uri = zones_list_df[zones_list_df['Zone.name'] == new_zone].iloc[0]['href']

**Note:** using `.iloc[0]` helps pull out the string in the `href` field, even though this is a dataframe with a single row and column.

The next step is to install the zone using the `install` RAD method. 

In this example we want to pass an `sc_profile.xml` file with the system configuration of the new zone with things like it's hostname and IP address so we won't have to set these at first boot. To do this we'll point to a copy of the XML file local to the server, and use the `'options:' []` field in the JSON text we're sending along with the `PUT` method. 

Note that this method will actually wait until the full zone is installed, so this may take a while:

In [32]:
json_body = {'options': ['-c', '/export/home/demo/sc_profile.xml']}

single_zone_install = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/install'.format(single_zone_uri), json_body)

print(single_zone_install.status_code, '\n')
print(json.loads(single_zone_install.text))

200 

{'status': 'success', 'payload': {'code': 'NONE', 'str': '', 'stdout': "The following ZFS file system(s) have been created:\n    rpool/VARSHARE/zones/test2\nProgress being logged to /var/log/zones/zoneadm.20201216T182115Z.test2.install\n       Image: Preparing at /system/zones/test2/root.\n\n Install Log: /system/volatile/install.21831/install_log\n AI Manifest: /tmp/manifest.xml.Vi1bca\n  SC Profile: /export/home/demo/sc_profile.xml\n    Zonename: test2\nInstallation: Starting ...\n\n        Creating IPS image\n         Startup: Retrieving catalog 'solaris' ... Done\n         Startup: Retrieving catalog 'cr-labtech' ... Done\n         Startup: Retrieving catalog 'solarisstudio' ... Done\n         Startup: Caching catalogs ... Done\n         Startup: Finished processing linked images.\n         Startup: Refreshing catalog 'solaris' ... Done\n         Startup: Refreshing catalog 'cr-labtech' ... Done\n         Startup: Refreshing catalog 'solarisstudio' ... Done\n        Installin

As you can see this returns the full text output to `stdout` that you would normally get on the CLI. To get a better look at it and make it more readable we print it out:

In [33]:
print(json.loads(single_zone_install.text)['payload']['stdout'])

The following ZFS file system(s) have been created:
    rpool/VARSHARE/zones/test2
Progress being logged to /var/log/zones/zoneadm.20201216T182115Z.test2.install
       Image: Preparing at /system/zones/test2/root.

 Install Log: /system/volatile/install.21831/install_log
 AI Manifest: /tmp/manifest.xml.Vi1bca
  SC Profile: /export/home/demo/sc_profile.xml
    Zonename: test2
Installation: Starting ...

        Creating IPS image
         Startup: Retrieving catalog 'solaris' ... Done
         Startup: Retrieving catalog 'cr-labtech' ... Done
         Startup: Retrieving catalog 'solarisstudio' ... Done
         Startup: Caching catalogs ... Done
         Startup: Finished processing linked images.
         Startup: Refreshing catalog 'solaris' ... Done
         Startup: Refreshing catalog 'cr-labtech' ... Done
         Startup: Refreshing catalog 'solarisstudio' ... Done
        Installing packages from:
            solaris
                origin:  http://ipkg.us.oracle.com/solaris11/

Now the zone is installed this should also be reflected in the status of the zone:

In [34]:
zones_status = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(single_zone_uri))
print(json.loads(zones_status.text)['payload']['Zone']['state'])

installed


We can also pull in the full list of zones and it should now be part of this list too:

In [35]:
zones_list = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(zone_uri))
zones_list_df = pd.io.json.json_normalize(json.loads(zones_list.text)['payload'])
zones_list_df

,href,Zone.auxstate,Zone.brand,Zone.id,Zone.uuid,Zone.name,Zone.state
0,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test1,[],solaris,-1,c5f26d28-0a9d-4c20-a2f3-eb2225de7f7e,test1,installed
1,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test2,[],solaris,-1,None,test2,installed


To now fully check everything is working we can boot the new zone:

In [36]:
json_body = {}

single_zone_boot = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/boot'.format(single_zone_uri), json_body)

print(single_zone_boot.status_code, '\n')
print(json.loads(single_zone_boot.text))

200 

{'status': 'success', 'payload': {'code': 'NONE', 'str': '', 'stdout': None, 'stderr': None}}


Finally we can check if the zone is indeed up and running:

In [37]:
zones_list = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(zone_uri))
zones_list_df = pd.io.json.json_normalize(json.loads(zones_list.text)['payload'])
zones_list_df

,href,Zone.auxstate,Zone.brand,Zone.id,Zone.uuid,Zone.name,Zone.state
0,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test1,[],solaris,-1,c5f26d28-0a9d-4c20-a2f3-eb2225de7f7e,test1,installed
1,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test2,[],solaris,21,None,test2,running


Now we're done we can halt, uninstall, and delete the new zone with the following requests. Note we're printing out the `status_code` for each request to verify we end up with all `200`s:

In [38]:
json_body = {}
json_body_uninstall = {"options": ['-F']}
json_body_delete = {'name': new_zone}

single_zone_halt = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/halt'.format(single_zone_uri), json_body)
print('Halting the zone:', single_zone_halt.status_code, '\n')

single_zone_uninstall = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/uninstall'.format(single_zone_uri), json_body_uninstall)
print('Uninstalling the zone:', single_zone_uninstall.status_code, '\n')

single_zone_delete = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/delete'.format(zone_manager_uri), json_body_delete)
print('Deleting the zone:', single_zone_delete.status_code, '\n')

Halting the zone: 200 

Uninstalling the zone: 200 

Deleting the zone: 200 



And now we're back to where we started:

In [39]:
zones_list = rad_rest_request('GET', s, server_connection_info, '{}?_rad_detail'.format(zone_uri))
zones_list_df = pd.io.json.json_normalize(json.loads(zones_list.text)['payload'])
zones_list_df

,href,Zone.auxstate,Zone.brand,Zone.id,Zone.uuid,Zone.name,Zone.state
0,api/com.oracle.solaris.rad.zonemgr/1.7/Zone/test1,[],solaris,-1,c5f26d28-0a9d-4c20-a2f3-eb2225de7f7e,test1,installed


This end the notebook. Find the full list of RAD methods in the Appendix below.

---
## Appendix

### List of Methods for each Interface

- **ZoneManager** — URI: `ZoneManager/` — Manage the zones on this system, for example create and delete zones. The RAD/REST methods on this interface are:
    - **PUT create**
    - **PUT delete**
    - **PUT createConfig**
    - **PUT importConfig**
    - **PUT connectRemote**
    - **PUT initEvacuate**
    - **PUT evacuate**
    - **PUT cancelEvacuate**
- **Zone** — URI: `Zone/` — For operations that affect a single zone, it represents an individual zone. All zone configuration and administrative actions are represented in this interface. The RAD/REST methods on this interface are:
    - **PUT cancelConfig**
    - **PUT exportConfig**
    - **PUT update**
    - **PUT editConfig**
    - **PUT commitConfig**
    - **PUT configIsLive**
    - **PUT configIsStale**
    - **PUT addResource**
    - **PUT reloadConfig**
    - **PUT removeResources**
    - **PUT getResources**
    - **PUT getResourceProperties**
    - **PUT setResourceProperties**
    - **PUT clearResourceProperties**
    - **PUT apply**
    - **PUT attach**
    - **PUT boot**
    - **PUT clone**
    - **PUT detach**
    - **PUT halt**
    - **PUT install**
    - **PUT mark**
    - **PUT migrate**
    - **PUT move**
    - **PUT rename**
    - **PUT ready**
    - **PUT reboot**
    - **PUT savecore**
    - **PUT shutdown**
    - **PUT suspend**
    - **PUT uninstall**
    - **PUT verify**
    - **PUT prepareForMigration**
    - **PUT initMigration**